## Plan to focus on Metabolism -> Biosynthesis of building blocks {2 members} 
- [x] G7496 --> Mo Cofactor (Skip, Cyrus has incorporated it)
- [x] EG11531 --> cardiolipins (TODO: Update lipids in homeostatic functions)

In [1]:
import numpy as np
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")

In [2]:
# load basal new model
time = '400'
date = '2025-01-30'
experiment = 'NEW_updated_bad_rxn'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba_basal = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent_basal = dill.load(f)
f.close()

In [84]:
# get commonly stored variables
metabolism_basal = agent_basal['ecoli-metabolism-redux-classic']
stoichiometry = metabolism_basal.stoichiometry.copy()
reaction_names = metabolism_basal.reaction_names
kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions
fba_new_reaction_ids = metabolism_basal.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism_basal.parameters['fba_reaction_ids_to_base_reaction_ids']
binary_kinetic_idx = metabolism_basal.binary_kinetic_idx
metabolites = metabolism_basal.metabolite_names

S = stoichiometry.copy()
S = pd.DataFrame(S, index=metabolites, columns=reaction_names)
homeostatic_count = pd.DataFrame(fba_basal["homeostatic_metabolite_counts"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
homeostatic = pd.DataFrame(fba_basal["target_homeostatic_dmdt"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
maintenance = pd.DataFrame(fba_basal["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]

FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

## load variables from track_reaction_usage.ipynb

In [85]:
%store -r df_reaction_flux df_reaction_track df_gene_usage unused_kinetic_gene

## Define functions for testing

In [111]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, 
                          add_homeostatic_demand = None, force_reaction = None):
    # update exchanges
    uptake = metabolism_basal.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal
    
    exchange_molecules = metabolism_basal.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules
    
    # update stoichiometry
    reaction_names = metabolism_basal.reaction_names.copy()
    kinetic_reaction_ids = metabolism_basal.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba_basal["target_kinetic_fluxes"], columns=metabolism_basal.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism_basal.metabolite_names.copy()
    homeostatic = pd.DataFrame(fba_basal["target_homeostatic_dmdt"], columns=metabolism_basal.homeostatic_metabolites).loc[24, :].copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism_basal.counts_to_molar.asNumber()
    
    S_new = stoichiometry.copy()
    
    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)
        print(S_new.shape)
    
    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)
        
        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)
        print(S_new.shape)
    
    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)
        
        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
                
    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]
                
    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)
        
        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1
            
    print(f'Force Reaction ID is:',force_reaction_idx)
    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=list(dict(homeostatic).keys()),
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.   
            solver=cp.GLOP) #SCS. ECOS
    print(f'Optimal objective function reached is: {solution.objective}')
    return solution.velocities, reaction_names, S_new, metabolites, kinetic

## Focus on gene G7496 --> RXN0-6254 first

In [112]:
# print G7496 reaction stoichiometry
G7496_S = S.loc[:, ['RXN0-6254']]
G7496_S = G7496_S.loc[~np.all(G7496_S == 0, axis=1), :]
print(G7496_S.index)

met_of_interest = ['CPD-15870[c]', 'CPD-23445[c]']
G7496_S, rxns_of_interest = get_subset_S(S, met_of_interest)
G7496_S = G7496_S.loc[:, ~np.all(G7496_S == 0, axis=0)]

df_reaction_flux.loc[rxns_of_interest,:]

Index(['CPD-15870[c]', 'CPD-23445[c]', 'CTP[c]', 'PPI[c]', 'PROTON[c]'], dtype='object')


,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
RXN-16457,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-21563,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-21601,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-24143,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN0-262,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN0-6254,0.0,0.0,0.0,-0.0,-0.0,False,True,True


In [94]:
G7496_S

,RXN-16457,RXN-21563,RXN-21601,RXN-24143,RXN0-262,RXN0-6254
CPD-15870[c],-1,1,-1,1,-1,-1
CPD-23445[c],0,0,0,0,0,1


In [95]:
G7496_S_expanded = S.loc[:, rxns_of_interest]
G7496_S_expanded = G7496_S_expanded.loc[~np.all(G7496_S_expanded == 0, axis=1), :]
G7496_S_expanded

,RXN-16457,RXN-21563,RXN-21601,RXN-24143,RXN0-262,RXN0-6254
AMP[c],0,1,0,1,0,0
CPD-15870[c],-1,1,-1,1,-1,-1
CPD-15873[c],0,0,1,0,0,0
CPD-15874[c],1,0,0,0,0,0
CPD-23445[c],0,0,0,0,0,1
CPD-26685[c],0,0,0,-1,0,0
CPD-3[c],0,-1,0,0,0,0
CPD-4[c],-1,0,-1,0,0,0
CPD-582[c],0,0,0,0,1,0
CPD-8122[c],0,-1,0,0,0,0


In [96]:
met_of_interest = ['CPD-15870[c]', 'CPD-15873[c]', 'CPD-15874[c]',
       'CPD-23445[c]', 'CPD-26685[c]', 'CPD-3[c]', 'CPD-4[c]', 'CPD-582[c]',
       'CPD-8122[c]']
G7496_S_expanded2, rxns_of_interest2 = get_subset_S(S, met_of_interest)
df_reaction_flux.loc[rxns_of_interest2,:]

,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
ABC-19-RXN,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-16456,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-16457,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-21562,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN-21563,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-21601,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-24141,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-24142,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-24143,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-8342,0.0,0.0,0.0,-0.0,-0.0,False,True,False


In [97]:
G7496_S_expanded2

,ABC-19-RXN,RXN-16456,RXN-16457,RXN-21562,RXN-21563,RXN-21601,RXN-24141,RXN-24142,RXN-24143,RXN-8342,RXN-8344,RXN0-262,RXN0-2787,RXN0-2787 (reverse),RXN0-6254,RXN0-7198
CPD-15870[c],0,0,-1,0,1,-1,0,0,1,0,0,-1,0,0,-1,0
CPD-15873[c],0,1,0,-1,0,1,0,0,0,0,0,0,0,0,0,0
CPD-15874[c],0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
CPD-23445[c],0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
CPD-26685[c],0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0
CPD-3[c],1,0,0,0,-1,0,-1,0,0,0,0,0,0,0,0,0
CPD-4[c],0,0,-1,0,0,-1,0,0,0,1,-1,0,1,-1,0,1
CPD-582[c],0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
CPD-8122[c],0,0,0,0,-1,0,-1,0,0,0,1,0,0,0,0,0


In [98]:
rxns_of_interest3 = ['RXN-8340','RXN-17809','RXN-8342','RXN0-7198','RXN0-2787', 'RXN0-2787 (reverse)']
df_reaction_flux.loc[rxns_of_interest3,:]

,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
RXN-8340,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-17809,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN-8342,0.0,0.0,0.0,-0.0,-0.0,False,True,False
RXN0-7198,0.0,0.0,0.0,0.0,0.0,False,False,False
RXN0-2787,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-2787 (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False


In [99]:
metabolism_basal.homeostatic_metabolites

array(['2-3-DIHYDROXYBENZOATE[c]', '2-KETOGLUTARATE[c]', '2-PG[c]',
       '2K-4CH3-PENTANOATE[c]', '4-AMINO-BUTYRATE[c]',
       '4-hydroxybenzoate[c]', 'ACETOACETYL-COA[c]', 'ACETYL-COA[c]',
       'ACETYL-P[c]', 'ADENINE[c]', 'ADENOSINE[c]', 'ADP-D-GLUCOSE[c]',
       'ADP[c]', 'AMP[c]', 'ANTHRANILATE[c]', 'APS[c]', 'ARG[c]',
       'ASN[c]', 'ATP[c]', 'BIOTIN[c]', 'CA+2[c]', 'CAMP[c]',
       'CARBAMYUL-L-ASPARTATE[c]', 'CARBON-DIOXIDE[c]', 'CDP[c]',
       'CHORISMATE[c]', 'CIS-ACONITATE[c]', 'CIT[c]', 'CL-[c]', 'CMP[c]',
       'CO+2[c]', 'CO-A[c]', 'CPD-12115[c]', 'CPD-12261[p]',
       'CPD-12575[c]', 'CPD-12819[c]', 'CPD-12824[c]', 'CPD-13469[c]',
       'CPD-2961[c]', 'CPD-8260[c]', 'CPD-9956[c]', 'CPD0-939[c]',
       'CTP[c]', 'CYS[c]', 'CYTIDINE[c]', 'CYTOSINE[c]', 'D-ALA-D-ALA[c]',
       'D-SEDOHEPTULOSE-7-P[c]', 'DAMP[c]', 'DATP[c]', 'DCTP[c]',
       'DEOXY-RIBOSE-5P[c]', 'DEOXYADENOSINE[c]', 'DEOXYGUANOSINE[c]',
       'DGMP[c]', 'DGTP[c]', 'DI-H-OROTATE[c]',
       '

In [100]:
homeostatic_count

2-3-DIHYDROXYBENZOATE[c]     96206.0
2-KETOGLUTARATE[c]          246091.0
2-PG[c]                      63998.0
2K-4CH3-PENTANOATE[c]        96206.0
4-AMINO-BUTYRATE[c]         211931.0
                              ...   
NA+[p]                       69714.0
OXYGEN-MOLECULE[p]           69714.0
FE+3[p]                      69714.0
CA+2[p]                      69714.0
Pi[p]                        69714.0
Name: 24, Length: 172, dtype: float64

In [101]:
force_reaction = ['RXN-8340']
add_reaction = {
    'TEMP - transport': {
        'CPD-19179[c]': -1,
        'CPD-19179[p]': 1,
    },
    'TEMP - transport (reverse)': {
        'CPD-19179[c]': 1,
        'CPD-19179[p]': -1,
    }
}
objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.000001}
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights, add_reaction=add_reaction,
                                            force_reaction=force_reaction
)    

ValueError: 'CPD-19179[p]' is not in list

In [102]:
df_reaction_flux.loc['RXN0-7206-CPD-4//CPD-4.13.',:]

basal                  0.0
acetate                0.0
rich                   0.0
anaerobic_basal       -0.0
anaerobic_acetate     -0.0
anaerobic_rich       False
is_new               False
is_kinetic           False
Name: RXN0-7206-CPD-4//CPD-4.13., dtype: object

## EG11531 -> 2.1.1.79-RXN (6)

In [113]:
# print EG11531 reaction stoichiometry
rxns_of_interest = ['2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.', 
                    '2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.',
                    '2.1.1.79-RXN-CPD-18367/S-ADENOSYLMETHIONINE//CPD-18371/ADENOSYL-HOMO-CYS/PROTON.67.',
                    '2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67.',
                    '2.1.1.79-RXN-CPD-18392/S-ADENOSYLMETHIONINE//CPD-18405/ADENOSYL-HOMO-CYS/PROTON.67.',
                    '2.1.1.79-RXN-CPD-18403/S-ADENOSYLMETHIONINE//CPD-18404/ADENOSYL-HOMO-CYS/PROTON.67.']

EG11531_S = S.loc[:, rxns_of_interest]
EG11531_S = EG11531_S.loc[~np.all(EG11531_S == 0, axis=1), :]
print(EG11531_S.index)

met_of_interest = ['CPD-18361[c]', 'CPD-18373[c]']
EG11531_S, rxns_of_interest = get_subset_S(S, met_of_interest)
EG11531_S = EG11531_S.loc[:, ~np.all(EG11531_S == 0, axis=0)]
EG11531_S.T
df_reaction_flux.loc[rxns_of_interest,:]

Index(['ADENOSYL-HOMO-CYS[c]', 'CPD-18361[c]', 'CPD-18362[c]', 'CPD-18367[c]',
       'CPD-18369[c]', 'CPD-18371[c]', 'CPD-18372[c]', 'CPD-18373[c]',
       'CPD-18392[c]', 'CPD-18403[c]', 'CPD-18404[c]', 'CPD-18405[c]',
       'CPD-18406[c]', 'PROTON[c]', 'S-ADENOSYLMETHIONINE[c]'],
      dtype='object')


,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,0.0,0.0,0.0,-0.0,-0.0,False,True,True
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
PGPPHOSPHA-RXN-CPD-18359/WATER//CPD-18361/Pi.30.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-5223-CPD-18361/ATP//CPD-18359/ADP/PROTON.36.,0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7189-CPD-18361//CPD-18361.21.,0.0,0.0,0.0,0.0,0.0,False,False,False
RXN0-7189-CPD-18361//CPD-18361.21. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,0.0,0.0,0.0,0.0,False,False,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),0.0,0.0,0.0,-0.0,-0.0,False,False,False
RXN0-7272-CPD-18367/GLYCEROL//CPD-18361/ETHANOL-AMINE.44.,0.0,0.0,0.0,0.0,-0.0,False,False,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,0.0,0.0,0.0,-0.0,-0.0,False,False,False


In [71]:
met_of_interest = ['CPD-18361[c]', 'CPD-18373[c]']
EG11531_S, rxns_of_interest = get_subset_S(S, met_of_interest)

## feels like the key is homeostatic objective function for this one

In [56]:
met_of_interest = ['CPD-8260[c]','CPD-12824[c]']
S_temp, rxns_of_interest = get_subset_S(S, met_of_interest)
S_temp.T
df_reaction_flux.loc[rxns_of_interest,:]

,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,is_new,is_kinetic
CARDIOLIPSYN-RXN-CPD-8260//CPD-12824/GLYCEROL.29.,430.7850,15.6075,1390.8375,4.170000e+02,16.0,False,False,False
PGPPHOSPHA-RXN-CPD-12821/WATER//CPD-8260/Pi.29.,0.0000,0.0000,0.0000,1.280662e-15,0.0,False,False,False
RXN0-5223-CPD-8260/ATP//CPD-12821/ADP/PROTON.35.,0.0000,0.0000,0.0000,-0.000000e+00,-0.0,False,False,False
RXN0-7189-CPD-8260//CPD-8260.19.,0.0000,0.0000,0.0000,0.000000e+00,0.0,False,False,False
RXN0-7189-CPD-8260//CPD-8260.19. (reverse),0.0000,0.0000,0.0000,-0.000000e+00,-0.0,False,False,False
RXN0-7272-CPD-12819/GLYCEROL//CPD-8260/ETHANOL-AMINE.43.,2412.5225,15.6075,7789.0400,2.334000e+03,1585.0,False,False,False
TRANS-RXN0-573-CPD-8260/ATP/WATER//CPD-8260/ADP/Pi/PROTON.43.,0.0000,0.0000,0.0000,-0.000000e+00,-0.0,False,False,False


In [57]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [59]:
sim_flux.loc[rxns_of_interest,:]

,flux,is_new
CARDIOLIPSYN-RXN-CPD-8260//CPD-12824/GLYCEROL.29.,417.0,Old Reactions
PGPPHOSPHA-RXN-CPD-12821/WATER//CPD-8260/Pi.29.,0.0,Old Reactions
RXN0-5223-CPD-8260/ATP//CPD-12821/ADP/PROTON.35.,-0.0,Old Reactions
RXN0-7189-CPD-8260//CPD-8260.19.,0.0,Old Reactions
RXN0-7189-CPD-8260//CPD-8260.19. (reverse),-0.0,Old Reactions
RXN0-7272-CPD-12819/GLYCEROL//CPD-8260/ETHANOL-AMINE.43.,2334.0,Old Reactions
TRANS-RXN0-573-CPD-8260/ATP/WATER//CPD-8260/ADP/Pi/PROTON.43.,-0.0,Old Reactions


fix 2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.

In [115]:
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            add_homeostatic_demand=add_homeostatic_demand
)

Force Reaction ID is: None
Optimal objective function reached is: 21782.197102609778


In [116]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [118]:
met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18371[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18367/S-ADENOSYLMETHIONINE//CPD-18371/ADENOSYL-HOMO-CYS/PROTON.67.,-0.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,-0.0,Old Reactions,False
TRANS-RXN0-573-CPD-18373/ATP/WATER//CPD-18373/ADP/Pi/PROTON.45.,-0.0,Old Reactions,False


fix 2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.

CPD-18406 to be homeostatically demanded

In [122]:
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18406[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            add_homeostatic_demand=add_homeostatic_demand
)

Force Reaction ID is: None
Optimal objective function reached is: 21799.345813614746


In [123]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [124]:
met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18406[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.,87.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7189-CPD-18406//CPD-18406.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18406//CPD-18406.21. (reverse),-0.0,Old Reactions,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,0.0,Old Reactions,False
RXN0-7272-CPD-18372/GLYCEROL//CPD-18406/ETHANOL-AMINE.44.,13.0,Old Reactions,False
TRANS-RXN0-573-CPD-18373/ATP/WATER//CPD-18373/ADP/Pi/PROTON.45.,-0.0,Old Reactions,False


fix 2.1.1.79-RXN-CPD-18367/S-ADENOSYLMETHIONINE//CPD-18371/ADENOSYL-HOMO-CYS/PROTON.67.   --> ends up making CPD-18373, so converges with first 2.1.1.79 reaction

In [119]:
# try removing first 2.1.1.79 reaction to see if this one will be used
remove_reaction = ['2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.']
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            remove_reaction=remove_reaction,
                                            add_homeostatic_demand=add_homeostatic_demand
)

Force Reaction ID is: None
Optimal objective function reached is: 21782.892069444653


In [120]:
sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

In [121]:
met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18371[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18367/S-ADENOSYLMETHIONINE//CPD-18371/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,300.0,Old Reactions,False
TRANS-RXN0-573-CPD-18373/ATP/WATER//CPD-18373/ADP/Pi/PROTON.45.,-0.0,Old Reactions,False


Yes, it becomes used in absence of 2.1.1.79 (1). i.e. in absence of CPD-18361. 

fix 2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67. --> ends up making CPD-18406

In [135]:
# try removing second 2.1.1.79 reaction to see if this one will be used
remove_reaction = ['2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.']
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18406[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            remove_reaction=remove_reaction,
                                            add_homeostatic_demand=add_homeostatic_demand
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18372[c]', 'CPD-18406[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

Force Reaction ID is: None
Optimal objective function reached is: 21799.34575282088


,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67.,100.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7189-CPD-18406//CPD-18406.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18406//CPD-18406.21. (reverse),-0.0,Old Reactions,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,-0.0,Old Reactions,False
RXN0-7272-CPD-18372/GLYCEROL//CPD-18406/ETHANOL-AMINE.44.,100.0,Old Reactions,False
TRANS-RXN0-573-CPD-18373/ATP/WATER//CPD-18373/ADP/Pi/PROTON.45.,-0.0,Old Reactions,False


Yes, it becomes used in absence of 2.1.1.79 (2). i.e. in absence of CPD-18362. 

fix 2.1.1.79-RXN-CPD-18392/S-ADENOSYLMETHIONINE//CPD-18405/ADENOSYL-HOMO-CYS/PROTON.67. --> include CPD-18405 in homeostatic func

In [132]:
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18406[c]', 'CPD-18405[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            add_homeostatic_demand=add_homeostatic_demand
)


sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18372[c]', 'CPD-18406[c]', 'CPD-18405[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

Force Reaction ID is: None
Optimal objective function reached is: 21814.259664121215


,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.,13.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67.,87.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18392/S-ADENOSYLMETHIONINE//CPD-18405/ADENOSYL-HOMO-CYS/PROTON.67.,100.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7189-CPD-18405//CPD-18405.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18405//CPD-18405.21. (reverse),-0.0,Old Reactions,False
RXN0-7189-CPD-18406//CPD-18406.21.,0.0,Old Reactions,False


fix 2.1.1.79-RXN-CPD-18403/S-ADENOSYLMETHIONINE//CPD-18404/ADENOSYL-HOMO-CYS/PROTON.67. --> ends up making CPD-18405

In [134]:
# try removing fifth 2.1.1.79 reaction to see if this one will be used
remove_reaction = ['2.1.1.79-RXN-CPD-18392/S-ADENOSYLMETHIONINE//CPD-18405/ADENOSYL-HOMO-CYS/PROTON.67.']
add_homeostatic_demand = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18406[c]', 'CPD-18405[c]']
solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            remove_reaction=remove_reaction,
                                            add_homeostatic_demand=add_homeostatic_demand
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids 
        else 'TEMP' if id in add_reaction.keys() 
        else 'Old Reactions' 
            for id in test_reaction_names
    ]
}, index=test_reaction_names)

met_of_interest = ['CPD-18373[c]', 'CPD-18375[c]', 'CPD-18372[c]', 'CPD-18404[c]']
S_new = pd.DataFrame(S_new, index=test_metabolites, columns=test_reaction_names)
S_met, rxns  = get_subset_S(S_new, met_of_interest)
rxns
rxn_flux = sim_flux.loc[rxns]
rxn_flux['kinetic'] = [test_kinetic[r] if r in kinetic_reaction_ids else False for r in rxn_flux.index]
rxn_flux

Force Reaction ID is: None
Optimal objective function reached is: 21814.49126148075


,flux,is_new,kinetic
2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,300.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67.,13.0,New Reactions,87.0
2.1.1.79-RXN-CPD-18403/S-ADENOSYLMETHIONINE//CPD-18404/ADENOSYL-HOMO-CYS/PROTON.67.,100.0,New Reactions,87.0
CARDIOLIPSYN-RXN-CPD-18373//CPD-18375/GLYCEROL.30.,100.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21.,0.0,Old Reactions,False
RXN0-7189-CPD-18373//CPD-18373.21. (reverse),-0.0,Old Reactions,False
RXN0-7272-CPD-18371/GLYCEROL//CPD-18373/ETHANOL-AMINE.44.,0.0,Old Reactions,False
RXN0-7272-CPD-18372/GLYCEROL//CPD-18406/ETHANOL-AMINE.44.,13.0,Old Reactions,False
RXN0-7272-CPD-18404/GLYCEROL//CPD-18405/ETHANOL-AMINE.44.,100.0,Old Reactions,False
TRANS-RXN0-573-CPD-18373/ATP/WATER//CPD-18373/ADP/Pi/PROTON.45.,-0.0,Old Reactions,False


Yes, it becomes used in absence of 2.1.1.79 (5). i.e. in absence of CPD-18392. 